In [1]:
from sklearn.datasets import load_iris

In [2]:
iris=load_iris()

In [8]:
import pandas as pd
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower']=iris.target
df['flower']=df['flower'].apply(lambda x: iris.target_names[x])
df

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

        flower  
0       setosa  
1       setosa  
2       setosa  
3       setosa  
4       setosa  
..         ...  
145  virginica  
146  virginica  
147  virginica  
148  virginica  
149  virginica  

[150 rows x 5 columns]

## Normal Approach

In [27]:
from sklearn.svm import SVC

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.3)

In [47]:
model=SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)

SVC(C=30, gamma='auto')

In [48]:
model.score(X_test,y_test)

0.8888888888888888

In [49]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [50]:
x1=cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)
np.average(x1)

0.9800000000000001

In [51]:
x2=cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),iris.data,iris.target,cv=5)
np.average(x2)

0.9666666666666668

In [52]:
x3=cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)
np.average(x3)

0.9733333333333334

In [53]:
kernels=['rbf','linear']
C=[1,10,20]
for i in kernels:
    for j in C:
        crossval_score=cross_val_score(SVC(kernel=i,C=j,gamma='auto'),iris.data,iris.target,cv=5)
        print(f"{i}_{j} : {np.average(crossval_score)}")

rbf_1 : 0.9800000000000001
rbf_10 : 0.9800000000000001
rbf_20 : 0.9666666666666668
linear_1 : 0.9800000000000001
linear_10 : 0.9733333333333334
linear_20 : 0.9666666666666666


In [54]:
## all of the above methods fail when it comes to the case of large data + large no of iterations

## GridSearchCV

In [55]:
from sklearn.model_selection import GridSearchCV

In [56]:
clf=GridSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20],##This is specific only to a given limit and not like range[1:50] as it tries various permutations and combinations.
    'kernel':['rbf','linear']
},cv=5,return_train_score=False)
clf.fit(iris.data,iris.target)
df1=pd.DataFrame(clf.cv_results_)
df1

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0       0.000613      0.000501         0.000000        0.000000       1   
1       0.000520      0.000470         0.000231        0.000461       1   
2       0.000467      0.000582         0.000201        0.000401      10   
3       0.000201      0.000401         0.000420        0.000515      10   
4       0.000501      0.000448         0.000000        0.000000      20   
5       0.000411      0.000504         0.000000        0.000000      20   

  param_kernel                         params  split0_test_score  \
0          rbf      {'C': 1, 'kernel': 'rbf'}           0.966667   
1       linear   {'C': 1, 'kernel': 'linear'}           0.966667   
2          rbf     {'C': 10, 'kernel': 'rbf'}           0.966667   
3       linear  {'C': 10, 'kernel': 'linear'}           1.000000   
4          rbf     {'C': 20, 'kernel': 'rbf'}           0.966667   
5       linear  {'C': 20, 'kernel': 'linear'}           1.000000   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0                1.0           0.966667           0.966667                1.0   
1                1.0           0.966667           0.966667                1.0   
2                1.0           0.966667           0.966667                1.0   
3                1.0           0.900000           0.966667                1.0   
4                1.0           0.900000           0.966667                1.0   
5                1.0           0.900000           0.933333                1.0   

   mean_test_score  std_test_score  rank_test_score  
0         0.980000        0.016330                1  
1         0.980000        0.016330                1  
2         0.980000        0.016330                1  
3         0.973333        0.038873                4  
4         0.966667        0.036515                5  
5         0.966667        0.042164                6

In [58]:
df1[['param_C','param_kernel','mean_test_score']]

param_C param_kernel  mean_test_score
0       1          rbf         0.980000
1       1       linear         0.980000
2      10          rbf         0.980000
3      10       linear         0.973333
4      20          rbf         0.966667
5      20       linear         0.966667

In [66]:
print(dir(clf))
print(clf.best_score_)
print(clf.best_params_)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_is_fitted', '_check_n_features', '_check_refit_for_multimetric', '_estimator_type', '_format_results', '_get_param_names', '_get_tags', '_more_tags', '_pairwise', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_run_search', '_validate_data', 'best_estimator_', 'best_index_', 'best_params_', 'best_score_', 'classes_', 'cv', 'cv_results_', 'decision_function', 'error_score', 'estimator', 'fit', 'get_params', 'inverse_transform', 'multimetric_', 'n_features_in_', 'n_jobs', 'n_splits_', 'param_grid', 'pre_dispatch', 'predict', 'predict_log_pro

## Randomized GridSearchCv

In [67]:
## When there is a lot of computation involved and the sample need not go through all of it then we use RandomizedSearchCV

In [80]:
from sklearn.model_selection import RandomizedSearchCV
clf1=RandomizedSearchCV(SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear']
},cv=5,n_iter=2,return_train_score=False)
clf1.fit(iris.data,iris.target)
df2=pd.DataFrame(clf1.cv_results_)
df2

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_kernel  \
0       0.000200      0.000400           0.0002          0.0004       linear   
1       0.000944      0.000507           0.0000          0.0000       linear   

  param_C                         params  split0_test_score  \
0      20  {'kernel': 'linear', 'C': 20}                1.0   
1      10  {'kernel': 'linear', 'C': 10}                1.0   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0                1.0                0.9           0.933333                1.0   
1                1.0                0.9           0.966667                1.0   

   mean_test_score  std_test_score  rank_test_score  
0         0.966667        0.042164                2  
1         0.973333        0.038873                1

In [81]:
df2[['param_kernel','param_C','mean_test_score']]

param_kernel param_C  mean_test_score
0       linear      20         0.966667
1       linear      10         0.973333

In [82]:
print(dir(clf1))

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_is_fitted', '_check_n_features', '_check_refit_for_multimetric', '_estimator_type', '_format_results', '_get_param_names', '_get_tags', '_more_tags', '_pairwise', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_run_search', '_validate_data', 'best_estimator_', 'best_index_', 'best_params_', 'best_score_', 'classes_', 'cv', 'cv_results_', 'decision_function', 'error_score', 'estimator', 'fit', 'get_params', 'inverse_transform', 'multimetric_', 'n_features_in_', 'n_iter', 'n_jobs', 'n_splits_', 'param_distributions', 'pre_dispatch', 'predict

In [83]:
print(clf1.best_score_)

0.9733333333333334


In [84]:
print(clf1.best_params_)

{'kernel': 'linear', 'C': 10}
